In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv(r"C:\Users\indra\OneDrive\Desktop\filtered_data_folder\filtered_branches.csv")

In [3]:
data.isnull().sum()

DateCreated       0
MoveDate          0
MoveType          0
MoveSize          0
HourlyRate        0
CrewSize          0
Trucks            0
Branch            0
Source            0
Current_Status    0
BookStatus        0
Lag_Days          0
weekday           0
is_weekend        0
season            0
dtype: int64

In [4]:
data = data.dropna(subset=['MoveDate', 'Lag_Days'])

In [5]:
data.isnull().sum()

DateCreated       0
MoveDate          0
MoveType          0
MoveSize          0
HourlyRate        0
CrewSize          0
Trucks            0
Branch            0
Source            0
Current_Status    0
BookStatus        0
Lag_Days          0
weekday           0
is_weekend        0
season            0
dtype: int64

In [6]:
data.shape

(318407, 15)

In [7]:
data.head()

,DateCreated,MoveDate,MoveType,MoveSize,HourlyRate,CrewSize,Trucks,Branch,Source,Current_Status,BookStatus,Lag_Days,weekday,is_weekend,season
0,09-01-2022 15:59,22-02-2022 00:00,Local,Apt 1 Bedroom,169.0,3,1,Fort Worth,MobileAMSMainLead,Cancelled,No Book,43,1,0,Winter
1,10-01-2022 09:37,26-02-2022 00:00,Short Haul,House 3 Bedroom,159.0,2,1,Houston,HAExactMatch,Cancelled,No Book,46,5,1,Winter
2,10-01-2022 12:24,08-02-2022 00:00,Local,House 3 Bedroom,159.0,3,1,Phoenix,GMB,Completed,Booked,28,1,0,Winter
3,10-01-2022 13:43,27-01-2022 00:00,Long Distance,House 3 Bedroom,119.0,2,1,Phoenix,HAMarketMatch,Cancelled,No Book,16,3,0,Winter
4,10-01-2022 15:45,11-01-2022 00:00,Local,Storage,199.0,3,1,Houston,pricingpagegoogledec2019CC,Completed,Booked,0,1,0,Winter


In [8]:
data.tail()

,DateCreated,MoveDate,MoveType,MoveSize,HourlyRate,CrewSize,Trucks,Branch,Source,Current_Status,BookStatus,Lag_Days,weekday,is_weekend,season
318402,27-11-2020 16:05,05-12-2020,Local,Apt 2 Bedroom or more,159.0,3,1,Houston,pricingpagegoogledec2019,Cancelled,No Book,7,5,1,Winter
318403,30-11-2020 11:45,19-12-2020,Local,Apt 1 Bedroom,159.0,3,1,Phoenix,OTHER,Completed,Booked,18,5,1,Winter
318404,05-09-2020 14:35,05-10-2020,Long Distance,Apt 1 Bedroom,219.0,4,1,Houston,A-PPC,Cancelled,No Book,29,0,0,Fall
318405,16-11-2020 15:38,23-11-2020,Local,House 4 Bedroom or more,299.0,5,2,Fort Worth,MobileAMSMainLead,Completed,Booked,6,0,0,Fall
318406,30-11-2020 18:16,01-12-2020,Local,Apt 1 Bedroom,129.0,2,1,Houston,pricingpagegoogledec2019coupon,Cancelled,No Book,0,1,0,Winter


In [9]:
data.dtypes

DateCreated        object
MoveDate           object
MoveType           object
MoveSize           object
HourlyRate        float64
CrewSize            int64
Trucks              int64
Branch             object
Source             object
Current_Status     object
BookStatus         object
Lag_Days            int64
weekday             int64
is_weekend          int64
season             object
dtype: object

In [10]:
from sklearn.preprocessing import LabelEncoder
# List of categorical columns (object type)
categorical_cols = ['DateCreated', 'MoveDate', 'MoveType', 'MoveSize', 
                    'Branch', 'Source', 'Current_Status', 'BookStatus','season']

# Apply Label Encoding
label_encoders = {} 

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])  # Transform categorical values to numerical
    label_encoders[col] = le  # Store encoder for later use (inverse transform if needed)

# Display the transformed DataFrame
print(data.head())

   DateCreated  MoveDate  MoveType  MoveSize  HourlyRate  CrewSize  Trucks  \
0        71017      5365         7         0       169.0         3       1   
1        79661      6244        13         5       159.0         2       1   
2        79718      1735         7         5       159.0         3       1   
3        79747      6434        10         5       119.0         2       1   
4        79793      2452         7         9       199.0         3       1   

   Branch  Source  Current_Status  BookStatus  Lag_Days  weekday  is_weekend  \
0       1     204               4           1        43        1           0   
1       2     172               4           1        46        5           1   
2       4     155               5           0        28        1           0   
3       4     173               4           1        16        3           0   
4       2     303               5           0         0        1           0   

   season  
0       3  
1       3  
2       3  
3 

In [11]:
data.shape

(318407, 15)

In [12]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
import pickle

# Assuming 'data' is your DataFrame and multiple columns are used as predictors
time_column = "MoveDate"
target_columns = ["HourlyRate", "Trucks"]  # Adjust for multiple time series

# Convert date column to datetime format
data[time_column] = pd.to_datetime(data[time_column])
data.set_index(time_column, inplace=True)

# Train the VAR model
train_data = data[target_columns]

# Define and fit the model
model = VAR(train_data)
model_fitted = model.fit(maxlags=15, ic='aic')  # Select lag order using AIC

# Save the model
with open("var_model.pkl", "wb") as f:
    pickle.dump(model_fitted, f)

print("VAR model trained and saved successfully.")


C:\Users\indra\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\indra\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


VAR model trained and saved successfully.


In [13]:
# Load the saved VAR model
with open("var_model.pkl", "rb") as f:
    model_fitted = pickle.load(f)

# Define forecasting horizon
forecast_steps = 30  # Adjust as needed

# Get the last observed values for forecasting
input_data = train_data.values[-model_fitted.k_ar:]  # Use last k_ar observations

# Make predictions
forecast = model_fitted.forecast(y=input_data, steps=forecast_steps)

# Convert forecast to DataFrame
forecast_index = pd.date_range(start=train_data.index[-1], periods=forecast_steps + 1, freq="D")[1:]
forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=target_columns)

# Compute evaluation metrics (e.g., RMSE)
from sklearn.metrics import mean_squared_error

for col in target_columns:
    actual = data[col].iloc[-forecast_steps:]  # Get actual values
    predicted = forecast_df[col].values
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    print(f"RMSE for {col}: {rmse}")


RMSE for HourlyRate: 48.146863201571605
RMSE for Trucks: 0.17958120698047214


In [14]:
#!pip install mxnet-cu112 
#!pip install gluonts
#!pip install --upgrade mxnet==1.6.0
#!pip install "gluonts[torch]"

In [15]:
from sklearn.model_selection import train_test_split

target_col = 'HourlyRate'  # Change this to your actual target column
X = data.drop(columns=[target_col])  # Features
y = data[target_col]  # Target variable

# Split the dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the splits
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")

X_train shape: (254725, 13), X_test shape: (63682, 13)
y_train shape: (254725,), y_test shape: (63682,)


In [16]:
# Install necessary packages (uncomment if needed)
# !pip install lightgbm pandas numpy scikit-learn

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pickle

# Prepare LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)

# Define model parameters
params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "verbose": -1
}

# Train the LightGBM model
model = lgb.train(params, train_data, num_boost_round=100)

# Save the model
with open("lightgbm_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model training complete. Saved as 'lightgbm_model.pkl'")

# Load the model for validation on train data
with open("lightgbm_model.pkl", "rb") as f:
    model = pickle.load(f)

# Make predictions on train data
y_train_pred = model.predict(X_train)

# Evaluate the model on train data
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(f"RMSE on train data: {rmse_train}")


# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE on test data: {rmse}")


Model training complete. Saved as 'lightgbm_model.pkl'
RMSE on train data: 16.035977428273423
RMSE on test data: 15.698927915096128


In [17]:
# Compute absolute differences between actual and predicted values
test_results = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})
test_results["Difference"] = abs(test_results["Actual"] - test_results["Predicted"])

# Get the top 20 highest differences
top_20_high_diff = test_results.nlargest(20, "Difference")

# Display the results
print("Top 20 highest difference values between Predicted vs Actual:")
print(top_20_high_diff)


Top 20 highest difference values between Predicted vs Actual:
                               Actual   Predicted  Difference
MoveDate                                                     
1970-01-01 00:00:00.000005394   955.0  356.887827  598.112173
1970-01-01 00:00:00.000006447   100.0  494.820781  394.820781
1970-01-01 00:00:00.000006019   540.0  205.342479  334.657521
1970-01-01 00:00:00.000002090   100.0  417.170047  317.170047
1970-01-01 00:00:00.000000190   100.0  413.570937  313.570937
1970-01-01 00:00:00.000001815   500.0  220.346680  279.653320
1970-01-01 00:00:00.000003313   609.0  360.807208  248.192792
1970-01-01 00:00:00.000006023    20.0  265.209255  245.209255
1970-01-01 00:00:00.000000152   400.0  640.644396  240.644396
1970-01-01 00:00:00.000007133   159.0  395.241804  236.241804
1970-01-01 00:00:00.000004459   111.0  320.126238  209.126238
1970-01-01 00:00:00.000002075   398.0  195.352351  202.647649
1970-01-01 00:00:00.000006905   153.0  353.115001  200.115001
1970-01-

In [18]:
# Install necessary packages
#!pip install pandas numpy scikit-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle

# Train RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the model
with open("random_forest_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model training complete. Saved as 'random_forest_model.pkl'")

# Load the model for testing
with open("random_forest_model.pkl", "rb") as f:
    model = pickle.load(f)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE on test data: {rmse}")

# How to Run:
# 1. Ensure 'your_data.csv' exists with relevant feature columns and 'Target'.
# 2. Run this script to train and save the model.
# 3. Load the saved model to make predictions and evaluate on test data.


Model training complete. Saved as 'random_forest_model.pkl'
RMSE on test data: 14.744360495473963


In [19]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

# Sample dataset (replace with your actual dataset)
data = pd.read_csv(r"C:\Users\indra\OneDrive\Desktop\filtered_data_folder\filtered_branches.csv")
# Convert to DataFrame
df = pd.DataFrame(data)

# Identify categorical columns
categorical_cols = ["MoveType", "MoveSize", "Branch", "Source", "Current_Status", "BookStatus", "season"]
#df["MoveDate"] = pd.to_datetime(df["MoveDate"]).map(pd.Timestamp.toordinal)


# Create label encoders dictionary
label_encoders = {}

# Fit and transform each categorical column
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # Fit and transform dataset
    label_encoders[col] = le  # Store label encoder

# Save label encoders to a pickle file
with open("label_encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)

print("Label encoders saved successfully as 'label_encoders.pkl'.")


Label encoders saved successfully as 'label_encoders.pkl'.


In [20]:
with open("label_encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)

print("Label encoders saved as 'label_encoders.pkl'")

Label encoders saved as 'label_encoders.pkl'


In [21]:
def predict_new_data(new_data):
    # Convert input to DataFrame
    df_input = pd.DataFrame([new_data])

    # Convert date columns to ordinal format (if used in training)
    df_input["DateCreated"] = pd.to_datetime(df_input["DateCreated"]).map(pd.Timestamp.toordinal)

    # Remove MoveDate if it wasn't used during training
    df_input.drop(columns=["MoveDate"], inplace=True, errors="ignore")

    # Encode categorical variables
    for col in df_input.select_dtypes(include=["object"]).columns:
        if col in label_encoders:
            try:
                df_input[col] = label_encoders[col].transform(df_input[col])
            except ValueError:
                print(f"Error: Unseen category in {col}: {df_input[col].values}")
                return None  # Stop prediction if an unseen category is found

    # Ensure the input columns match the training feature set
    df_input = df_input[X_train.columns]

    # Predict output
    prediction = model.predict(df_input)
    return prediction[0]
# Example Input (Fixed Format)
new_input = {
    "DateCreated": "2025-03-11 11:51:00",  # Assuming this is a date feature
    "MoveDate": "2025-03-26	 00:00",  # Assuming this is a date feature
    "MoveType": "Local",  # Categorical
    "MoveSize": "Apt 1 Bedroom",  # Numeric
    "CrewSize": 4,  # Numeric
    "Trucks": 1,  # Numeric
    "Branch": "Houston",  # Categorical
    "Source": "MobileAMSMainLead",  # Categorical
    "Current_Status": "Completed",  # Categorical
    "BookStatus": "Booked", 
    "Lag_Days": 3, # Categorical
    "weekday": 5,  # Numeric
    "is_weekend": 0,  # Binary (0 = No, 1 = Yes)
    "season": "Spring"  # Categorical
}

predicted_value = predict_new_data(new_input)
print(f"Predicted value: {predicted_value}")

# Save label encoders as pickle file



Predicted value: 205.48
